In [289]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()





In [290]:
t_train = pd.read_csv("train.csv")
t_test = pd.read_csv("test.csv")

In [297]:
### Checking which columns have NA values
#t_train.isnull().any()  returns Boolean list of columns with NA or not


t_train.columns[t_train.isnull().any()]


Index([], dtype='object')

In [292]:
## Number of NA values in columns with NAs


t_train[['Age','Cabin','Embarked']].isnull().sum()

Age         177
Cabin       687
Embarked      2
dtype: int64

In [293]:
# Nullify Cabin, Drop  Embarked NA rows, Give median to Age NA values


t_train.drop('Cabin', axis=1, inplace=True)




In [294]:
#Drop Embarked NA rows
t_train = t_train[t_train['Embarked'].notnull()]

In [295]:
#Check if  Embarked has na

t_train['Embarked'].isnull().sum()

0

In [298]:
## Give Age na values median value

med = t_train['Age'].median()


t_train['Age'] = t_train['Age'].fillna(value=med)



In [299]:
#Family unit or not

def nuking (row):
    if row['SibSp'] >= 1 and row['Parch']>=1:
      return "FM"
    else:
        return "Not"
    


t_train['NUKEFM_or_Not'] = t_train.apply (lambda row: nuking (row),axis=1)

In [300]:
########## create a variable that designates if they are a Miss or Not
# Turn Age column into float 

#def f(x):
#    try:
#        return np.float(x)
#    except:
#        return np.nan

#t_train['Age'] = t_train['Age'].apply(f)


def missing (row):
    if row['Age'] >= 18 and row['Name'].find("Miss") != -1:
      return "Miss"
    else:
      return "Not"
    

    #if row['Age'] >= 18 and row['Name'].find("Miss") != -1:
     
#_train['Age']= pd.to_numeric(t_train['Age'], errors='coerce').fillna(0)


t_train['Miss_or_Not'] = t_train.apply (lambda row: missing (row),axis=1)
#t_train.apply (lambda row: missing (row),axis=1)





In [301]:
### # Create new column  family_size
def sizing(row):
    len = row['SibSp'] + row['Parch'] + 1
    return len
    
t_train['family_size']=t_train.apply (lambda row: sizing (row),axis=1)

In [302]:
## turn Survived into Categorical Variable
def surving (row):
    if row['Survived'] == 1 :
      return "Yes"
    else:
        return "No"
    


t_train['Survived'] = t_train.apply (lambda row: surving (row),axis=1)

In [303]:
## turn PClass into Categorical Variable
def classing (row):
    if row['Pclass'] == 1 :
      return "First"
    elif row['Pclass'] == 2:
        return "Second"
    else:
        return"Third"
    


t_train['Pclass'] = t_train.apply (lambda row: classing (row),axis=1)

In [304]:
t_train['Pclass'], uniclass = pd.factorize(t_train['Pclass'], sort=True)
uniclass                            

Index(['First', 'Second', 'Third'], dtype='object')

In [305]:

##
t_train['Survived'], uniques = pd.factorize(t_train['Survived']) 

t_train['Sex'], unisex = pd.factorize(t_train['Sex'])
t_train['Embarked'], uniembar = pd.factorize(t_train['Embarked'])
t_train['Miss_or_Not'], unimiss = pd.factorize(t_train['Miss_or_Not'])
t_train['NUKEFM_or_Not'], uninuke = pd.factorize(t_train['NUKEFM_or_Not'])

In [335]:
t_train.info()
uniclass


<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 11 columns):
Survived         889 non-null int64
Pclass           889 non-null int64
Sex              889 non-null int64
Age              889 non-null float64
SibSp            889 non-null int64
Parch            889 non-null int64
Fare             889 non-null float64
Embarked         889 non-null int64
NUKEFM_or_Not    889 non-null int64
Miss_or_Not      889 non-null int64
family_size      889 non-null int64
dtypes: float64(2), int64(9)
memory usage: 83.3 KB


Index(['First', 'Second', 'Third'], dtype='object')

In [355]:
t_train.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,NUKEFM_or_Not,Miss_or_Not,family_size
0,0,2,0,22.0,1,0,7.2500,0,0,0,2
1,1,0,1,38.0,1,0,71.2833,1,0,0,2
2,1,2,1,26.0,0,0,7.9250,0,0,1,1
3,1,0,1,35.0,1,0,53.1000,0,0,0,2
4,0,2,0,35.0,0,0,8.0500,0,0,0,1
5,0,2,0,28.0,0,0,8.4583,2,0,0,1
6,0,0,0,54.0,0,0,51.8625,0,0,0,1
7,0,2,0,2.0,3,1,21.0750,0,1,0,5
8,1,2,1,27.0,0,2,11.1333,0,0,0,3
9,1,1,1,14.0,1,0,30.0708,1,0,0,2


In [308]:
# First remove Name and Ticket collumns
t_train.drop('Name', axis=1, inplace=True)
t_train.drop('Ticket', axis=1, inplace=True)
t_train.drop('PassengerId', axis=1, inplace=True)

In [309]:
import xgboost as xgb

In [336]:

#Create training and test datasets
X_train = t_train.drop("Survived", axis = 1)
y_train = t_train['Survived'].values
#y_train = y_train.reshape(-1, 1)
#X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state=42)
X_test = t_test



In [337]:
y_train.shape

###
#X_train.shape

(889,)

In [338]:
#Convert the training and testing sets into DMatrixes
#DMatrix is the recommended class in xgboost.

DM_train = xgb.DMatrix(data = X_train, 
                       label = y_train)  
#DM_test =  xgb.DMatrix(t_test)

In [345]:

#Parameters for grid search

gbm_param_grid = {
     'colsample_bytree': np.linspace(0.5, 1, 5),
     'n_estimators':[100, 200],
     'max_depth': [5, 8, 10, 11, 15],      
}

#Instantiate the classifier
gbm = xgb.XGBClassifier()

In [347]:


#grid_mse = GridSearchCV(estimator = gbm, param_grid = gbm_param_grid, scoring = 'roc_auc',cv = 5, verbose = 1)
grid_mse2 = GridSearchCV(estimator = gbm, param_grid = gbm_param_grid, scoring = 'roc_auc',cv = 5, verbose = 1)

In [344]:
grid_mse.fit(X_train, y_train)
print("Best parameters found: ",grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x114640828>>
Traceback (most recent call last):
  File "/Users/georgesericcolbert/anaconda/lib/python3.6/site-packages/xgboost/core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


Best parameters found:  {'colsample_bytree': 0.75, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.625}
Lowest RMSE found:  0.936288146952


[Parallel(n_jobs=1)]: Done 1250 out of 1250 | elapsed:  2.0min finished


In [348]:
grid_mse2.fit(X_train, y_train)
print("Best parameters found: ",grid_mse2.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse2.best_score_)))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found:  {'colsample_bytree': 0.5, 'max_depth': 15, 'n_estimators': 100}
Lowest RMSE found:  0.933472084237


[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:   24.6s finished


In [395]:
t_test = pd.read_csv("test.csv")

In [398]:
### Checking which columns have NA values
#t_train.isnull().any()  returns Boolean list of columns with NA or not


#t_test.columns[t_test.isnull().sum()]
t_test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [397]:
# Drop Cabin
t_test.drop('Cabin', axis=1, inplace=True)

#Drop Fare NA rows
med = t_test['Fare'].median()


t_test['Fare'] = t_test['Fare'].fillna(value=med)

## Median Age

med = t_test['Age'].median()


t_test['Age'] = t_test['Age'].fillna(value=med)






In [399]:

t_test['NUKEFM_or_Not'] = t_test.apply (lambda row: nuking (row),axis=1)

t_test['Miss_or_Not'] = t_test.apply (lambda row: missing (row),axis=1)

t_test['family_size']=t_test.apply (lambda row: sizing (row),axis=1)




t_test['Pclass'] = t_test.apply (lambda row: classing (row),axis=1)

p_Id = t_test['PassengerId']
#  remove Name, Ticket & PassengerID collumns
t_test.drop('Name', axis=1, inplace=True)
t_test.drop('Ticket', axis=1, inplace=True)
t_test.drop('PassengerId', axis=1, inplace=True)

In [400]:


t_test['Pclass'], uniclass = pd.factorize(t_test['Pclass'], sort=True)




t_test['Sex'], unisex = pd.factorize(t_test['Sex'])
t_test['Embarked'], uniembar = pd.factorize(t_test['Embarked'])
t_test['Miss_or_Not'], unimiss = pd.factorize(t_test['Miss_or_Not'])
t_test['NUKEFM_or_Not'], uninuke = pd.factorize(t_test['NUKEFM_or_Not'])



In [401]:
t_test.head()


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,NUKEFM_or_Not,Miss_or_Not,family_size
0,2,0,34.5,0,0,7.8292,0,0,0,1
1,2,1,47.0,1,0,7.0000,1,0,0,2
2,1,0,62.0,0,0,9.6875,0,0,0,1
3,2,0,27.0,0,0,8.6625,1,0,0,1
4,2,1,22.0,1,1,12.2875,1,1,0,3


In [406]:
t_test.drop('Survived', axis=1, inplace=True)
t_test.drop('PassengerId', axis=1, inplace=True)

In [407]:
pred = grid_mse2.predict(t_test)



t_test['PassengerId'] = p_Id

t_test['Survived'] = pred


Kaggle_submit= t_test[['PassengerId', 'Survived']]


In [408]:
Kaggle_submit.shape

(418, 2)

In [409]:
Kaggle_submit.to_csv('PythonKagglesub9', index =False)